# Bedrock Knowledge Base Retrieval and Generation with Metadata Filtering

### Description:
This notebook demonstrates how to query and retrieve data from an Amazon Bedrock-powered knowledge base using different configurations, filters, and citation extraction. The steps include creating a query, retrieving responses, and printing the citations used for generating the results.


## 1. Load Configuration Variables

In [ ]:
# Load configuration variables from a JSON file to access knowledge base ID, account number, and guardrail info.
import json

with open("../Lab 1/variables.json", "r") as f:
    variables = json.load(f)

variables  # Display the loaded variables for confirmation

## 2. Set Up Required IDs and Model ARNs

In [ ]:
accountNumber=variables['accountNumber']   
knowledge_base_id = variables['kbFixedChunk']   
model_id = 'us.amazon.nova-pro-v1:0' 
model_arn = f"arn:aws:bedrock:us-west-2:{accountNumber}:inference-profile/{model_id}"


## 3. Configure Bedrock Client

In [ ]:
import boto3
import json
from typing import *

# Configure the Bedrock client
bedrock_agent_runtime = boto3.client('bedrock-agent-runtime', region_name="us-west-2")


## 4. Define Function to Retrieve and Generate Without Filters

In [ ]:
def retrieve_and_generate_without_filter(query, knowledge_base_id, model_arn):
    """
    Retrieves and generates a response based on the given query.

    Parameters:
    - query (str): The input query.
    - knowledge_base_id (str): The ID of the knowledge base.
    - model_arn (str): The ARN of the model.
    - one_group_filter (dict): The filter for the vector search configuration.

    Returns:
    - response: The response from the retrieve_and_generate method.
    """
    response = bedrock_agent_runtime.retrieve_and_generate(
        input={
            "text": query
        },
        retrieveAndGenerateConfiguration={
            "type": "KNOWLEDGE_BASE",
            "knowledgeBaseConfiguration": {
                'knowledgeBaseId': knowledge_base_id,
                "modelArn": model_arn,
                "retrievalConfiguration": {
                    "vectorSearchConfiguration": {
                        "numberOfResults": 5
                    }
                }
            }
        }
    )
    return response


## 5. Define Function to Retrieve and Generate With Filters

In [ ]:
def retrieve_and_generate_with_filter(query, knowledge_base_id, model_arn, metadata_filter):
    """
    Retrieves and generates a response based on the given query.

    Parameters:
    - query (str): The input query.
    - knowledge_base_id (str): The ID of the knowledge base.
    - model_arn (str): The ARN of the model.
    - one_group_filter (dict): The filter for the vector search configuration.

    Returns:
    - response: The response from the retrieve_and_generate method.
    """
    response = bedrock_agent_runtime.retrieve_and_generate(
        input={
            "text": query
        },
        retrieveAndGenerateConfiguration={
            "type": "KNOWLEDGE_BASE",
            "knowledgeBaseConfiguration": {
                'knowledgeBaseId': knowledge_base_id,
                "modelArn": model_arn,
                "retrievalConfiguration": {
                    "vectorSearchConfiguration": {
                        "numberOfResults": 5,
                        "filter": metadata_filter
                    }
                }
            }
        }
    )
    return response



## 6. Define Query

In [ ]:
query = "what was the % increase in sales?"

## 7. Retrieve Response Without Metadata Filter

In [ ]:
response_withoutMetadata=retrieve_and_generate_without_filter(query, knowledge_base_id, model_arn)
print(response_withoutMetadata['output']['text'])


## 8. Retrieve and Print Citations Without Metadata Filter

In [ ]:
# Extract citations used to generate the response
response_without_MD = response_withoutMetadata['citations'][0]['retrievedReferences']
print("# of citations or chunks used to generate the response: ", len(response_without_MD))

# Function to print citations or chunks of text retrieved
def citations_rag_print(response_ret):
    for num, chunk in enumerate(response_ret, 1):
        print(f'Chunk {num}: ', chunk['content']['text'], end='\n'*2)
        print(f'Chunk {num} Location: ', chunk['location'], end='\n'*2)
        print(f'Chunk {num} Metadata: ', chunk['metadata'], end='\n'*2)

# Print citations
citations_rag_print(response_without_MD)


## 9. Define Metadata Filter

The code below defines a metadata filter to narrow down the knowledge base search:
- Creates a complex filter using logical operators (andAll)
- The filter has two conditions that must both be true:
  1. docType must equal '10K Report'
  2. year must equal 2023
- This filter will limit retrieval to only chunks from 2023 10K reports
- The structure demonstrates how to build more complex queries with multiple conditions

This filter will be used to demonstrate selective retrieval from specific documents.

In [ ]:
# Define a metadata filter for advanced filtering based on specific conditions
one_group_filter= {
    "andAll": [
        {
            "equals": {
                "key": "docType",
                "value": '10K Report'
            }
        },
        {
            "equals": {
                "key": "year",
                "value": 2023
            }
        }
    ]
}


## 10. Retrieve Response With Metadata Filter

In [ ]:
# Use the function to retrieve a response with metadata filtering
response_with_Metadata = retrieve_and_generate_with_filter(query, knowledge_base_id, model_arn, one_group_filter)

# Print the response text
print(response_with_Metadata['output']['text'])


## 11. Retrieve and Print Citations With Metadata Filter

In [ ]:
# Extract citations used to generate the response with metadata filter
response_with_MD = response_with_Metadata['citations'][0]['retrievedReferences']
print("# of citations or chunks used to generate the response: ", len(response_with_MD))

# Print citations for the filtered response
citations_rag_print(response_with_MD)
